## 수정사항
#### 2021.06.06
* T_ej는 마지막 조인트 각도에 영향을 받음, 하지만 Q는 unkown이므로 특정하는게 불가능, T_ee로 전부 새로 학습할 것.

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="panda"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
ARM_FOLDER = ARM10_FOLDER

In [4]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

['20210920-194311', '20210920-233941', '20210921-032259', '20210921-072543', '20210921-111615', '20210921-150638', '20210921-192001', '20210921-232015', '20210922-033134', '20210922-074138']
['20210922-113744', '20210922-153855', '20210922-193840', '20210922-233844', '20210923-040005']
['20210923-084301', '20210923-090916', '20210923-092756', '20210923-094543', '20210923-101344', '20210923-103627', '20210923-110151', '20210923-112256', '20210923-114722', '20210923-121506']
['20210923-123816', '20210923-130206', '20210923-132926', '20210923-135036', '20210923-141543']


In [5]:
data_pairs_train_filtered = []
for dataset in DATASET_TRAIN_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_filtered)))        
        
        
data_pairs_test_filtered = []
for dataset in DATASET_TEST_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_filtered)))        

data_pairs_train_failmore = []
for dataset in DATASET_TRAIN_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("train fail set: {}".format(len(data_pairs_train_failmore)))        
        
        
data_pairs_test_failmore = []
for dataset in DATASET_TEST_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("test fail set: {}".format(len(data_pairs_test_failmore)))        

train set: 10003
test set: 5000
train fail set: 10038
test fail set: 5022


In [6]:
N_max_train = 10000
N_max_test = 5000

print("=== mix data ===")
data_pairs_train_mixed = mix_data_pairs(
    data_pairs_train_filtered, data_pairs_train_failmore, N_max_train)
data_pairs_test_mixed = mix_data_pairs(
    data_pairs_test_filtered, data_pairs_test_failmore, N_max_test)

data_pairs_train_filtered = data_pairs_train_filtered[:N_max_train]
data_pairs_test_filtered = data_pairs_test_filtered[:N_max_test]
data_pairs_train_failmore = data_pairs_train_failmore[:N_max_train]
data_pairs_test_failmore = data_pairs_test_failmore[:N_max_test]

print("="*20)
print("filtered data: {} / {}".format(
    len(data_pairs_train_filtered), 
    len(data_pairs_test_filtered)))
print("failmore data: {} / {}".format(
    len(data_pairs_train_failmore), 
    len(data_pairs_test_failmore)))
print("mixed data: {} / {}".format(
    len(data_pairs_train_mixed), 
    len(data_pairs_test_mixed)))

=== mix data ===
filtered success: 5000
success from failmore: 0
dataset fails: 5000
dataset all: 10000
filtered success: 2500
success from failmore: 0
dataset fails: 2500
dataset all: 5000
filtered data: 10000 / 5000
failmore data: 10000 / 5000
mixed data: 10000 / 5000


## Train

In [7]:
from pkg.planning.filtering.lattice_model.lattice_model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

    
def train_script(data_pairs_train, data_pairs_test, robot_model_root,
                 EPOCHS_S=0, EPOCHS_E=15, BATCH_SIZE=16, LOG_STEP=100):
    current_time = get_now()
    logpath = os.path.join(robot_model_root, current_time)
    try_mkdir(logpath)
    train_log_dir = os.path.join(logpath, 'train')
    test_log_dir = os.path.join(logpath, 'test')
    model_log_dir = os.path.join(logpath, 'model_{}/')
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    shutil.copy(os.path.join(
        PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model.py' ), logpath)
    print(f'Log path: {logpath}')
    
    N_train = len(data_pairs_train)
    N_test = len(data_pairs_test)
    random.shuffle(data_pairs_test)
    res_list = []
    
    for epoch in range(EPOCHS_S, EPOCHS_E):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        random.shuffle(data_pairs_train)
        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_train:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                train_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_test:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                test_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        model.save(model_log_dir.format(epoch + 1))

        print("")
        print("=================================================================")
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )
        print("=================================================================")
        print("")
        res_list.append((float(test_accuracy.result())*100, float(test_loss.result())))
    return res_list

## train filtered data

In [11]:
ROBOT_MODEL_ROOT_DIV = "{}-filtered".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_filtered, data_pairs_test_filtered

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=20, EPOCHS_E=30)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458/model_21/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458/model_21/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 21, Loss: 0.19432833790779114, Accuracy: 92.72500610351562, Test Loss: 0.3153074085712433, Test Accuracy: 86.9090576171875



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458/model_22/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458/model_29/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 29, Loss: 0.07243959605693817, Accuracy: 97.64999389648438, Test Loss: 0.41957345604896545, Test Accuracy: 85.62699890136719



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458/model_30/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-filtered/20211003-084458/model_30/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 30, Loss: 0.062171973288059235, Accuracy: 97.80500030517578, Test Loss: 0.5629350543022156, Test Accuracy: 85.84735107421875

/home/rnb/Projects/rnb-planning/model/latticized/panda-filtered acc/loss list:
1: 86.91 %, 0.315 
2: 86.88 %, 0.439 
3: 86.59 %, 0.429 
4: 87.15 %, 0.521 
5: 85.57 %, 0.465 
6: 82.99 %, 0.466 
7: 85.35 %, 0.422 
8: 86.57 %, 0.489 
9: 85.63 %, 0.42 
10: 85.85 %, 0.563 


```
16: 83.57 %, 1.115 
17: 87.54 %, 0.303 
18: 87.14 %, 0.319 
19: 86.62 %, 0.338 
20: 86.24 %, 0.373 
```

## failmore data

In [9]:
ROBOT_MODEL_ROOT_DIV = "{}-failmore".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_failmore, data_pairs_test_failmore

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108/model_1/assets



Epoch 1, Loss: 0.24209104478359222, Accuracy: 93.9800033569336, Test Loss: 0.08203324675559998, Test Accuracy: 96.13381958007812



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108/model_14/assets



Epoch 14, Loss: 0.03798489272594452, Accuracy: 98.63500213623047, Test Loss: 0.08285766839981079, Test Accuracy: 97.08534240722656



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108/model_15/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-failmore/20210923-150108/model_15/assets



Epoch 15, Loss: 0.02693922072649002, Accuracy: 99.04500579833984, Test Loss: 0.09839364141225815, Test Accuracy: 96.61457824707031

/home/rnb/Projects/rnb-planning/model/latticized/panda-failmore acc/loss list:
1: 96.13 %, 0.082 
2: 96.85 %, 0.077 
3: 96.48 %, 0.086 
4: 94.87 %, 0.105 
5: 96.94 %, 0.068 
6: 96.92 %, 0.077 
7: 96.29 %, 0.086 
8: 96.94 %, 0.076 
9: 97.06 %, 0.072 
10: 96.68 %, 0.096 
11: 95.64 %, 0.108 
12: 96.84 %, 0.088 
13: 96.88 %, 0.113 
14: 97.09 %, 0.083 
15: 96.61 %, 0.098 


## balanced data

In [10]:
ROBOT_MODEL_ROOT_DIV = "{}-balanced".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_mixed, data_pairs_test_mixed

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926/model_1/assets



Epoch 1, Loss: 0.16795837879180908, Accuracy: 94.5999984741211, Test Loss: 0.1439146250486374, Test Accuracy: 95.19230651855469



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926/model_14/assets



Epoch 14, Loss: 0.019831575453281403, Accuracy: 99.2750015258789, Test Loss: 0.18808874487876892, Test Accuracy: 95.56289672851562



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926/model_15/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/panda-balanced/20210923-151926/model_15/assets



Epoch 15, Loss: 0.019988996908068657, Accuracy: 99.44999694824219, Test Loss: 0.1989697813987732, Test Accuracy: 96.06370544433594

/home/rnb/Projects/rnb-planning/model/latticized/panda-balanced acc/loss list:
1: 95.19 %, 0.144 
2: 96.1 %, 0.119 
3: 95.6 %, 0.148 
4: 95.5 %, 0.136 
5: 96.14 %, 0.133 
6: 96.08 %, 0.146 
7: 88.43 %, 0.325 
8: 95.78 %, 0.134 
9: 96.1 %, 0.143 
10: 95.22 %, 0.185 
11: 94.43 %, 0.193 
12: 95.7 %, 0.16 
13: 95.94 %, 0.172 
14: 95.56 %, 0.188 
15: 96.06 %, 0.199 


# Train results - Panda

#### filtered: 20210923-144309
```
1: 75.59 %, 0.518 
2: 71.03 %, 0.745 
3: 87.61 %, 0.306 
4: 85.58 %, 0.447 
5: 85.2 %, 0.337 
6: 87.89 %, 0.292 
7: 86.23 %, 0.412 
8: 88.37 %, 0.32 
9: 88.21 %, 0.329 
10: 87.2 %, 0.327 
11: 87.54 %, 0.333 
12: 85.86 %, 0.544 
13: 87.93 %, 0.408 
14: 86.62 %, 0.363 
15: 85.85 %, 0.362 
```

#### failmore: 20210923-150108
```
1: 96.13 %, 0.082 
2: 96.85 %, 0.077 
3: 96.48 %, 0.086 
4: 94.87 %, 0.105 
5: 96.94 %, 0.068 
6: 96.92 %, 0.077 
7: 96.29 %, 0.086 
8: 96.94 %, 0.076 
9: 97.06 %, 0.072 
10: 96.68 %, 0.096 
11: 95.64 %, 0.108 
12: 96.84 %, 0.088 
13: 96.88 %, 0.113 
14: 97.09 %, 0.083 
15: 96.61 %, 0.098 
```

#### balanced: 20210923-151926
```
1: 95.19 %, 0.144 
2: 96.1 %, 0.119 
3: 95.6 %, 0.148 
4: 95.5 %, 0.136 
5: 96.14 %, 0.133 
6: 96.08 %, 0.146 
7: 88.43 %, 0.325 
8: 95.78 %, 0.134 
9: 96.1 %, 0.143 
10: 95.22 %, 0.185 
11: 94.43 %, 0.193 
12: 95.7 %, 0.16 
13: 95.94 %, 0.172 
14: 95.56 %, 0.188 
15: 96.06 %, 0.199 
```

## New train 2021-06
* **[IMPORTANT] 이전에 Tej 사용 틀린 점 보완 했음** 

### Panda
* 20210606-080139 : first balanced train trial, 96.6% 달성 / 이전 데이터와 collision boundary shape 차이일 수 있음
* 20210606-082302 : 이전 데이터와 전부 섞어서 collision boundary shape 차이 영향 확인 - 87% 가량, 영향 확실, 재수집 필요
* 20210606-191455 : 새로 모았어도 분리 학습 하니 시작부터 99%, 문제 있음
* 20210606-194148 : 새로 모은거 분리 학습, 테스트만 순수 기존 데이터로 하니 59%, 문제 확실
* 20210606-201525 : 새 데이터만 학습, 96%? 하지만 fail bias 돼 있음.
* 20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

### Indy7
* 20210607-224401 : 새 데이터만 학습, test set은 bias 제거함 ep12 79.5% 수준
* 20210610-052216 : 조인트 리밋 해제한 데이터만 학습, test set은 bias 제거, ep12 92% 수준

## Load & test

In [12]:
def get_tf_functions(model_root, model_folder, epoch_folder):
    # last_model = '20210606-204309' # panda
    # last_model = '20210610-052216' # indy
    # last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
    logpath = os.path.join(model_root, model_folder)

    # epochlist = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))
    # print(epochlist)
    # last_save = epochlist[-1]
    model_log_dir = os.path.join(logpath, epoch_folder)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    return model, inference, calc_loss

def test_script(data_pairs_test, model_root, model_folder, epoch_folder):
    BATCH_SIZE = 1
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()
    
    model, inference, calc_loss= get_tf_functions(model_root, model_folder, epoch_folder)

    grasp_img, arm_img, rh_mask, label = load_data(data_pairs_test[0])
    res = inference([np.array([grasp_img]), np.array([arm_img]), np.array([rh_mask])])


    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]

    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)

    TP_FN_FP_TN = (np.sum(np.logical_and(res_list, label_list)), np.sum(np.logical_and(np.logical_not(res_list), label_list)), 
                   np.sum(np.logical_and(res_list, np.logical_not(label_list))), np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list))))
        
    return label_list, res_list

# Test all

In [13]:
print("========= Filtered =========")
model_foler, epoch_folder = '20211003-081015', 'model_17'
filtered_res = []
filtered_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
# filtered_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
# filtered_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))

# print("========= Failmore =========")
# model_foler, epoch_folder = '20210923-150108', 'model_9'
# failmore_res = []
# failmore_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
# failmore_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
# failmore_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))

# print("========= Balanced =========")
# model_foler, epoch_folder = '20210923-151926', 'model_5'
# balanced_res = []
# balanced_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
# balanced_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
# balanced_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))

print("========= Filtered =========")
print("filtered:")
print_confusion_mat(*filtered_res[0])
# print("failmore:")
# print_confusion_mat(*filtered_res[1])
# print("balanced:")
# print_confusion_mat(*filtered_res[2])
# print("========================\n")

# print("========= Failmore =========")
# print("filtered:")
# print_confusion_mat(*failmore_res[0])
# print("failmore:")
# print_confusion_mat(*failmore_res[1])
# print("balanced:")
# print_confusion_mat(*failmore_res[2])
# print("========================\n")

# print("========= Balanced =========")
# print("filtered:")
# print_confusion_mat(*balanced_res[0])
# print("failmore:")
# print_confusion_mat(*balanced_res[1])
# print("balanced:")
# print_confusion_mat(*balanced_res[2])
# print("========================\n")

========= Filtered =========


test step - 5000/5000        
Test Loss: 0.3036595284938812 
Test Accuracy: 87.4 
TP / FN / ACC: 2949, 223, 92.97
FP / TN / ACC: 407, 1421, 77.74
PACC / NACC / TACC: 87.87, 86.44, 87.4


inference: 	54633.0 ms/5000 = 10.927 ms (10.325/14.416)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2949 	 	 223 	 	 92.97%
GN 	 407 	 	 1421 	 77.74%
AL 	 87.87% 	 86.44% 	 87.40%


### Test result - panda
```
inference: 	55039.0 ms/5000 = 11.008 ms (10.482/24.449)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2967 	 	 205 	 	 93.54%
GN 	 389 	 	 1439 	 78.72%
AL 	 88.41% 	 87.53% 	 88.12%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 203 	 	 11 	 	 94.86%
GN 	 1435 	 	 3351 	 70.02%
AL 	 12.39% 	 99.67% 	 71.08%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2338 	 	 162 	 	 93.52%
GN 	 709 	 	 1791 	 71.64%
AL 	 76.73% 	 91.71% 	 82.58%
========================

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1702 	 	 1470 	 	 53.66%
GN 	 177 	 	 1651 	 90.32%
AL 	 90.58% 	 52.90% 	 67.06%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 130 	 	 84 	 	 60.75%
GN 	 63 	 	 4723 	 98.68%
AL 	 67.36% 	 98.25% 	 97.06%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1336 	 	 1164 	 	 53.44%
GN 	 29 	 	 2471 	 98.84%
AL 	 97.88% 	 67.98% 	 76.14%
========================

========= Balanced =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 3109 	 	 63 	 	 98.01%
GN 	 833 	 	 995 	 54.43%
AL 	 78.87% 	 94.05% 	 82.08%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 209 	 	 5 	 	 97.66%
GN 	 295 	 	 4491 	 93.84%
AL 	 41.47% 	 99.89% 	 94.00%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2446 	 	 54 	 	 97.84%
GN 	 141 	 	 2359 	 94.36%
AL 	 94.55% 	 97.76% 	 96.10%
========================
```

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:5000,1]
    label_list = np.array(label_list)[:5000,1]
    loss_list = np.array(loss_list)[:5000]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

## test on shared array

In [ ]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-2
# Create an array in shared memory.
robot_type_p = sa.attach("shm://robot_type")
grasp_img_p = sa.attach("shm://grasp_img")
arm_img_p = sa.attach("shm://arm_img")
rh_mask_p = sa.attach("shm://rh_mask")
result_p = sa.attach("shm://result")
query_in = sa.attach("shm://query_in")
response_out = sa.attach("shm://response_out")
query_quit = sa.attach("shm://query_quit")

def query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch):
    grasp_img_p[:] = grasp_img_batch[:]
    arm_img_p[:] = arm_img_batch[:]
    rh_mask_p[:] = rh_mask_batch[:]
    query_in[0] = True
    while not response_out[0]:
        time.sleep(SERVER_PERIOD)
    response_out[0] = False
    return np.copy(result_p)

def quit_shared_server():
    query_quit[0] = True

In [ ]:
LOG_STEP = 100
N_test = len(data_pairs_test)

i_step = 0
data_batch, label_batch = [], []
result_list = []
label_list = []
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    data_batch.append([grasp_img, arm_img, rh_mask])
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
        arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
        rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
        label_batch = np.array(label_batch, dtype=np.int)
        result = query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch)
        result_list.append(result)
        label_list.append(label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

In [ ]:
result_all = np.array(result_list)
label_all = np.array(label_list)

In [ ]:
np.mean(np.round(result_all).astype(np.int) == label_all.astype(np.int), axis=0)

In [ ]:
quit_shared_server()